In [1]:
import glob
import pandas as pd
from autogluon.tabular import TabularDataset, TabularPredictor
from autogluon.common.features.feature_metadata import FeatureMetadata

In [2]:
model_name = "play_model_1-best"
training_files = 5
data_files = glob.glob('play_data/play_model_1-20240906*.tsv')
dfs = [pd.read_csv(file, sep='\t') for file in data_files[:training_files]]

train_data = TabularDataset(pd.concat(dfs, ignore_index=True))
train_data['play_pos'] = train_data['play_pos'].astype('category')

In [3]:
train_data.head()

,run_id,key,play_pos,pos,partner_winning,lead_trumped,tricks_won,trumps_seen,turn_seen,next_seen,...,throw_off_void,throw_off_long,throw_off_low,throw_off_next,tricks_min,tricks_max,tricks_avg,points_min,points_max,points_avg
0,77127b,9♦ J♦ 10♣ Q♣ Q♠,23,0,0,0,2,7,7,4,...,0,0,0,0,2,2,2.0,-1,-1,-1.0
1,77127b,9♦ J♦ 10♣ Q♠ Q♣,23,0,0,0,2,7,7,4,...,0,0,0,0,2,2,2.0,-1,-1,-1.0
2,77127b,10♣ 9♦ J♦ Q♠ Q♣,23,0,0,0,2,7,7,4,...,0,0,0,0,2,2,2.0,-1,-1,-1.0
3,77127b,10♣ 9♦ J♦ Q♣ Q♠,20,0,0,0,3,7,7,3,...,0,0,0,0,3,3,3.0,2,2,2.0
4,77127b,9♦ J♦ Q♠ 10♣ Q♣,23,0,0,0,2,7,7,4,...,0,0,0,0,2,2,2.0,-1,-1,-1.0


In [4]:
label = 'points_avg'
problem_type = 'regression'
presets = 'best_quality'
train_data[label].describe()

count    962167.000000
mean         -0.422773
std           1.914155
min          -4.000000
25%          -2.000000
50%          -1.000000
75%           1.000000
max           4.000000
Name: points_avg, dtype: float64

In [5]:
exclude = ['run_id', 'key']
outputs = ['tricks_min', 'tricks_max', 'tricks_avg', 'points_min', 'points_max', 'points_avg']
assert label in outputs
outputs.remove(label)

predictor = TabularPredictor(problem_type=problem_type, label=label, path=f"models/{model_name}")
predictor.fit(train_data.drop(columns=(exclude + outputs)), presets=presets)

Verbosity: 2 (Standard Logging)
=================== System Info ===================
AutoGluon Version:  1.1.1
Python Version:     3.11.9
Operating System:   Linux
Platform Machine:   x86_64
Platform Version:   #202405300957~1722440358~24.04~aa0a43f~dev-Ubuntu SMP PREEMPT_DY
CPU Count:          8
Memory Avail:       5.18 GB / 15.34 GB (33.8%)
Disk Space Avail:   116.76 GB / 452.95 GB (25.8%)
Presets specified: ['best_quality']
Setting dynamic_stacking from 'auto' to True. Reason: Enable dynamic_stacking when use_bag_holdout is disabled. (use_bag_holdout=False)
Stack configuration (auto_stack=True): num_stack_levels=1, num_bag_folds=8, num_bag_sets=1
DyStack is enabled (dynamic_stacking=True). AutoGluon will try to determine whether the input data is affected by stacked overfitting and enable or disable stacking as a consequence.
	This is used to identify the optimal `num_stack_levels` value. Copies of AutoGluon will be fit on subsets of the data. Then holdout validation data is used to 

(_ray_fit pid=854065) [1000]	valid_set's rmse: 0.889561


KeyboardInterrupt: 

(_ray_fit pid=854065) [2000]	valid_set's rmse: 0.84234
(_ray_fit pid=854364) [1000]	valid_set's rmse: 0.897125 [repeated 2x across cluster] (Ray deduplicates logs by default. Set RAY_DEDUP_LOGS=0 to disable log deduplication, or see https://docs.ray.io/en/master/ray-observability/ray-logging.html#log-deduplication for more options.)
(_ray_fit pid=854364) [2000]	valid_set's rmse: 0.84908 [repeated 2x across cluster]
(_ray_fit pid=854680) [1000]	valid_set's rmse: 0.891313 [repeated 2x across cluster]
(_ray_fit pid=854681) [1000]	valid_set's rmse: 0.897739


In [ ]:
test_data = TabularDataset(data_files[training_files])[:100000]
y_pred = predictor.predict(test_data[:1000])
y_pred.head(10)

In [ ]:
predictor.evaluate(test_data, silent=True)

In [ ]:
predictor.leaderboard(test_data)